In [ ]:
!pip install tensorflow_hub
! pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 4.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395187 sha256=082854eb38363c071324eb56d2e302d3cc91e5a44b12b7c1dc52ababbf20d344
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import keras
import pandas as pd
import pathlib
import PIL
import zipfile
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import time


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Testing

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/AML/proj-data/m1_test.csv")
train_data = pd.read_csv("/content/drive/MyDrive/AML/proj-data/m1_train.csv")
train_data

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
0,0,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714005,0.016932,2
1,1,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714002,0.011847,2
2,2,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714008,0.011847,2
3,3,2020-07-13,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,573390001,0.012186,1
4,4,2019-01-31,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,688873021,0.013542,1
...,...,...,...,...,...,...
247046,247046,2020-06-15,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,798579009,0.033881,2
247047,247047,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,888609001,0.025407,2
247048,247048,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,811871001,0.030492,2
247049,247049,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,838900002,0.030492,2


In [ ]:
test_cust_df = test_data.groupby('customer_id')['article_id'].agg(size = len, actual_buy= lambda x: list(x)).reset_index()
test_cust_df

,customer_id,size,actual_buy
0,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,1,[471714014]
1,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,1,[573390001]
2,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,5,"[608776020, 778064005, 772773002, 772773001, 7..."
3,00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bb...,21,"[740812007, 399223001, 554450025, 399223033, 7..."
4,0036414a83662a899a2e2c30854d8224932cc2e0d6f9c5...,9,"[860885002, 873019001, 806388005, 886966001, 8..."
...,...,...,...
12301,ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c...,3,"[757711001, 854004001, 640021012]"
12302,ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb...,1,[501722005]
12303,ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d...,5,"[698286007, 698286007, 699453001, 699453001, 6..."
12304,ffeb2a74486809e237cd05be056ca2e33b2141172e99d3...,4,"[685816008, 685816008, 685816008, 685816008]"


In [ ]:
# article similarity file
json_file = "/content/drive/MyDrive/AML/proj-data/nearest_neighbors2.json"

sim_df = pd.read_json(json_file)
sim_df

,similarity,master_pi,similar_pi
0,0.9656,710832001,709141001
1,0.9472,710832001,670944002
2,0.9434,710832001,746817001
3,0.9405,710832001,693604007
4,0.9398,710832001,847277001
...,...,...,...
945724,0.9598,901575001,764758004
945725,0.9567,901575001,935667002
945726,0.9553,901575001,935667001
945727,0.9530,901575001,694296001


In [ ]:
joined_df = pd.merge(train_data, sim_df, left_on='article_id', right_on='master_pi', how='left')
joined_df

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id,similarity,master_pi,similar_pi
0,0,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714005,0.016932,2,0.9650,471714005.0,689572001.0
1,0,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714005,0.016932,2,0.9584,471714005.0,800396008.0
2,0,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714005,0.016932,2,0.9555,471714005.0,630542002.0
3,0,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714005,0.016932,2,0.9549,471714005.0,399201020.0
4,0,2019-06-24,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,471714005,0.016932,2,0.9542,471714005.0,653114001.0
...,...,...,...,...,...,...,...,...,...
2214262,247050,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,671646003,0.016932,2,0.9131,671646003.0,905122001.0
2214263,247050,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,671646003,0.016932,2,0.9107,671646003.0,849073001.0
2214264,247050,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,671646003,0.016932,2,0.9085,671646003.0,829183002.0
2214265,247050,2020-06-23,fff3573d9131d15da6a46c1ca8f03b5d37e4f6b804171e...,671646003,0.016932,2,0.9072,671646003.0,706271023.0


In [ ]:
joined_set = joined_df.groupby(['customer_id'])['similar_pi'].agg(size = len, recc_buy= lambda x: list(x)).reset_index()
joined_set

,customer_id,size,recc_buy
0,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,27,"[689572001.0, 800396008.0, 630542002.0, 399201..."
1,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,9,"[666115002.0, 866657001.0, 463970010.0, 464277..."
2,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,162,"[695632006.0, 689389034.0, 689365001.0, 689365..."
3,00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bb...,756,"[674538002.0, 672758003.0, 636207005.0, 548110..."
4,0036414a83662a899a2e2c30854d8224932cc2e0d6f9c5...,288,"[357751002.0, 357751001.0, 818810001.0, 724216..."
...,...,...,...
12301,ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c...,72,"[758002008.0, 784785002.0, 736531003.0, 700714..."
12302,ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb...,9,"[518158001.0, 618854001.0, 475260003.0, 524869..."
12303,ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d...,153,"[589599022.0, 589599010.0, 745293008.0, 699082..."
12304,ffeb2a74486809e237cd05be056ca2e33b2141172e99d3...,117,"[736970001.0, 643547002.0, 806931002.0, 633932..."


In [ ]:
truth_df = pd.merge(test_cust_df, joined_set, left_on='customer_id', right_on='customer_id', how='left')
truth_df

,customer_id,size_x,actual_buy,size_y,recc_buy
0,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,1,[471714014],27,"[689572001.0, 800396008.0, 630542002.0, 399201..."
1,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,1,[573390001],9,"[666115002.0, 866657001.0, 463970010.0, 464277..."
2,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,5,"[608776020, 778064005, 772773002, 772773001, 7...",162,"[695632006.0, 689389034.0, 689365001.0, 689365..."
3,00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bb...,21,"[740812007, 399223001, 554450025, 399223033, 7...",756,"[674538002.0, 672758003.0, 636207005.0, 548110..."
4,0036414a83662a899a2e2c30854d8224932cc2e0d6f9c5...,9,"[860885002, 873019001, 806388005, 886966001, 8...",288,"[357751002.0, 357751001.0, 818810001.0, 724216..."
...,...,...,...,...,...
12301,ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c...,3,"[757711001, 854004001, 640021012]",72,"[758002008.0, 784785002.0, 736531003.0, 700714..."
12302,ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb...,1,[501722005],9,"[518158001.0, 618854001.0, 475260003.0, 524869..."
12303,ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d...,5,"[698286007, 698286007, 699453001, 699453001, 6...",153,"[589599022.0, 589599010.0, 745293008.0, 699082..."
12304,ffeb2a74486809e237cd05be056ca2e33b2141172e99d3...,4,"[685816008, 685816008, 685816008, 685816008]",117,"[736970001.0, 643547002.0, 806931002.0, 633932..."


In [ ]:
def apk2(actual, predicted):
  num_hits = 0.0

  for i,p in enumerate(actual):
    if p in predicted:
      num_hits += 1.0

  return num_hits / len(actual)

def mapk2(df):
  s = []
  for idx, row in df.iterrows():
    s.append(apk2(row['actual_buy'], row['recc_buy']))
  return np.mean(s)

In [ ]:
mapk2(truth_df)

0.01820986370174565
